![](https://design-style-guide.freecodecamp.org/downloads/freeCodeCamp-alternative.png)

# Exploring freeCodeCamp Gitter Messages

This notebook is a simple EDA and exploration of some general patterns in the freeCodeCamp Gitter chat log. Hopefully after reading this notebook you will have some good ideas of what you could do with this data yourself!

## Munging the data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
chat = pd.read_csv("../input/freecodecamp_casual_chatroom.csv", index_col=0)

In [ ]:
fig_kwargs = {'figsize': (12, 6), 'fontsize': 16}

In [ ]:
chat = chat.iloc[:, 1:]
chat.head(3)

For this notebook I will used the consolidated chat log, `freecodecamp_casual_chatroom.csv`. I generated this consolidated `csv` file by merging three distinct JSON files. The root files had slight overlap (a few days each), leading to a few duplicated messages being present in the unified file. Since this dataset handily comes equipped with an extremely accurate timestamp in the `sent` field (down to `ms`), we can see the exact overlapping records using the `pandas.Series.duplicated` method:

In [ ]:
chat.sent.duplicated().sum() / len(chat)

About 0.7 percent of messages were duplicated. We'll drop them before going any further.

In [ ]:
chat = chat.drop_duplicates()

In [ ]:
chat = chat.loc[~chat.sent.duplicated()]

With that out of the way, let's get exploring!

## Exploration

In [ ]:
chat['fromUser.displayName'].value_counts().head(10).plot.bar(**fig_kwargs, 
                                                              title="Top 10 Posters")

At first I assumed that Quincy Larson, the founder of freeCodeCamp, would be somewhere nearer to the top of the heap when it comes to posts on the public Gitter. Then I made this chart, and realized that there are people (the first one doesn't count because it's a robot) who made almost 150,000 posts over the course of the three years in this dataset. That works out to an average of over 100 chat posts *per day* for *three years*. Talk about an engaged usership.

In [ ]:
(pd.to_datetime(chat[chat['fromUser.displayName'] == "Serenity"].sent)
     .to_frame()
     .set_index('sent')
     .assign(n=0)
     .resample('M')
     .count()
     .plot.line(**fig_kwargs, title="Monthly Posts by A Top Poster"))

Here is the posting pattern for top poster `Serenity`. Their authorship continues to reach new heights!

In [ ]:
chat[chat['fromUser.staff'].notnull()].head()

The `staff` flag doesn't strike me as being very accurate; it only includes a few messages, all from 2015 or 2016.

In [ ]:
chat = chat.assign(sent=pd.to_datetime(chat.sent))

In [ ]:
ax = chat.sent.to_frame().assign(n=0).set_index('sent').resample('M').count().plot.line(
    **fig_kwargs, title="freeCodeCamp Gitter Chat Activity over Time"
)

(pd.to_datetime(chat[chat['fromUser.displayName'] == "Serenity"].sent)
     .to_frame()
     .set_index('sent')
     .assign(n=0)
     .resample('M')
     .count()
     .plot.line(ax=ax))

We see here that (unlike in certain chatrooms!) the amount of activity from even the most active poster on freeCodeCamp is nary a sliver comared to activity overall.

According to [Quincy Larson himself](https://twitter.com/ossia/status/959289114582396928), the sharp drop in activity in the chat in July of 2017 corresponds with the link to the chatroom being taken off of the new user registration onboarding flow. In other words, it was removed from the list of things that users were directed to immediately after they signed up. This reduced a _lot_ of traffic (probably mostly from low-volume posters). If I have the time, I would love to see what the impact of that change was in the type of messages (and level of noise) created in the chat. I suspect me-too type posts went down quite a bit, and that that effect would be measurable with a bit of NLP work.

Here's another way of representing the same information. What are the median, maximum, and average number of views that a message to the chat gets?

In [ ]:
import numpy as np
chat.groupby(pd.Grouper(key='sent', 
                        freq='M')).readBy.agg([np.median, 
                                               np.min, 
                                               np.max]).plot.line(**fig_kwargs, 
                                                                  title="freeCodeCamp Gitter Views Per Message")
ax = plt.gca()
ax.set_ylim([0, 100])

Given how busy the freeCodeCamp chat is, I'm surprised to learn that 0 message views are even a possibility. But, seemingly, it is. I expect that those messages must be buried in lots of other messages in low-activity times?

Note that I'm using the median, not the mean. Some messages perform two orders of magnitude better than a normal message. We can see why if we take a look at the high-rollers specifically:

In [ ]:
chat.loc[chat.readBy.sort_values(ascending=False).head(10).index].text

We see here that far and away the most heavily read messages are always those posted with the string `@/all` (and messages that come immediately after them, which users can't avoid reading). When a user does this, they ping _everyone else in the chatroom_. Depending on your notification settings, this can even send an email to users, including, potentially, users that have not been active in this chatroom for months, or even years.

`@/all` is really supposed to be reserved for important announcements from the staff. However, a quick scan of the top posts indicates that in a lot of cases it's used for things that are rather more banal (that Hello World in particular sticks out to me).

In [ ]:
chat.readBy.sort_values(ascending=False).head(10).mean()

To top 10 all-posts and nearby-posts averaged almost 4000 views. This is clearly _quite_ different from the median activity of 20 to 40 or so.

## Conclusion

There is lots more to do with this dataset. I haven't even taken a look at by far the most interesting feature of all: its textual NLP content. This dataset is a record of millions of messages from novice computer science learners and learning community members; what can you discover about the language they use, and/or about freeCodeCamp itself, by examing the text content of their speech? The possibilities are major!

Hopefully you enjoyed this simple EDA!